In [ ]:
pip install --upgrade sentence_transformers

In [ ]:
from torch.utils.data import DataLoader

from sentence_transformers import LoggingHandler, util
from sentence_transformers.cross_encoder import CrossEncoder
from sentence_transformers.cross_encoder.evaluation import CEBinaryClassificationEvaluator
from sentence_transformers.readers import InputExample
import pandas as pd
import csv
from sklearn.model_selection import train_test_split



# Tăng giới hạn kích thước dòng
csv.field_size_limit(10**7)

# Đường dẫn đến file CSV
csv_file = r"D:\Huan\Project\KHDL\finetune_data\rerank\pair_score.csv"

# Hàm đọc file CSV và chuyển đổi
def convert_csv_to_input_examples(csv_file):
    train_data = []
    with open(csv_file, "r", encoding="utf-8") as file:
        reader = csv.DictReader(file)
        for row in reader:
            sentence1 = row["sentence1"]
            sentence2 = row["sentence2"]
            score = float(row["score"])
            train_data.append(InputExample(texts=[sentence1, sentence2], label=score))
    return train_data




# Chuyển đổi CSV thành train_data
train_data = convert_csv_to_input_examples(csv_file)

train_samples, dev_samples = train_test_split(train_data, test_size=0.2, random_state=42)

In [ ]:
model = CrossEncoder("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2", num_labels=1)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
train_batch_size = 32
num_epochs = 1

In [ ]:
# We wrap train_samples (which is a List[InputExample]) into a pytorch DataLoader
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)


# We add an evaluator, which evaluates the performance during training
evaluator = CEBinaryClassificationEvaluator.from_input_examples(dev_samples, name="Quora-dev")

In [ ]:
import math
import csv
import logging
import math
import os
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1)  # 10% of train data for warm-up
#### Just some code to print debug information to stdout
logging.basicConfig(
    format="%(asctime)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S", level=logging.INFO, handlers=[LoggingHandler()]
)
logger = logging.getLogger(__name__)
logger.info(f"Warmup-steps: {warmup_steps}")

In [ ]:
# Train the model
model.fit(
    train_dataloader=train_dataloader,
    evaluator=evaluator,
    epochs=num_epochs,
    evaluation_steps=200,
    warmup_steps=warmup_steps,
    output_path=r'D:\Huan\Project\KHDL\model\rerank_model',
)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/3448 [00:00<?, ?it/s]

In [ ]:
from sentence_transformers import CrossEncoder
import torch

model = CrossEncoder("D:\Huan\Project\KHDL\model\rerank_model")
scores = model.predict([
    ("Triệu chứng của ung thư phổi?", "Dấu hiệu và triệu chứng \nNhững dấu hiệu và triệu chứng có thể là của ung thư phổi bao gồm:\nTriệu chứng về đường hô hấp: ho, ho ra máu, thở khò khè, khó thở\nTriệu chứng toàn thân: sụt cân, mệt mỏi, sốt, móng tay dùi trống"),
    ("Triệu chứng của ung thư phổi?", "Triệu chứng"),
])

In [ ]:
print(scores)

[0.96891564 0.49480754]
